# Introduction

This notebook demonstrates how to use the Vertex SDK to create video action recognition models using [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users).

Specifically, we will be training and evaluating a model that recognizes volleyball serves.

## Dataset

We will be using a [dataset](https://docs.google.com/spreadsheets/d/1MhT_ck2PhwYhPiKBE8v0z2TaFOgSLBgO1v562qObRvc/edit#gid=0) provided by Nagra. It consists of 23 videos, each containing around 50 - 200 serving events.

These videos are stored in a [GCS Bucket](https://console.cloud.google.com/storage/browser/aliz_action_recognition_poc/trunc_video?project=sharp-leaf-344111&pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&prefix=&forceOnObjectsSortingFiltering=false).

## Objective

In this notebook, we create an AutoML video action recognition model from a Python script.

The steps performed include:

- Create a Vertex `Dataset` resource.
- Train the model.


## Costs

This project uses billable components of Google Cloud:

* Vertex AI [(Vertex AI
pricing)](https://cloud.google.com/vertex-ai/pricing)
* Cloud Storage [(Cloud Storage
pricing)](https://cloud.google.com/storage/pricing)


# Setup

## Development environment setup

We'll need the following components:

0. ### GCPs:

- [Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).
- [GCP billing](https://cloud.google.com/billing/docs/how-to/modify-project)
- [Enablement of the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)


1. ### Basics:
- Git
- Python 3
- pipenv

2. ### SDKs:
- Cloud Storage SDK
- AI Platform SDK

3. ###  Data and video analytic libraries :
- pandas
- numpy
- matplotlib
- OpenCV
- [MoviePy](https://zulko.github.io/moviepy/) for video editing


Note: if we are running this notebook on Vertex AI Workbench, our environment already meets the requirements 1 and 2 above.

## Authenticate Google Cloud account (for local only)



**If we are using Vertex AI Notebooks**, your environment is already authenticated. Skip this step.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [1]:
# This provides access to our Cloud Storage bucket and lets us submit training jobs and prediction
# requests.
import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

# <font color='red'>Global Parameters</font>

<font color='red'> This section sets the global parameters for the downstream sections. Always run the cells in this section even if you're only running the later section (e.g. prediction)

### Project ID

In [2]:
PROJECT_ID = "sharp-leaf-344111"  # @param {type:"string"}

In [3]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)
else:
    print("Project ID:", PROJECT_ID)

Project ID: sharp-leaf-344111


In [26]:
# only run if it is local
! gcloud config set project $PROJECT_ID

Are you sure you wish to set property [core/project] to sharp-leaf-344111?

Do you want to continue (Y/n)?  ^C


Command killed by keyboard interrupt



### Region

We can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. To start with, we will be using  `europe-west4` as it is the closest to Nagra. The following regions are also available for Vertex AI:

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

Note: we may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services. (learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations))

In [4]:
REGION = "us-central1"  

### Prefix

All Vertex AI resources created will be prefixed by this preset prefix:

In [5]:
PREFIX = "volley_"

### GCS bucket

When we initialize the Vertex SDK for Python, we specify a Cloud Storage staging bucket, into which all data associated with our dataset and model resources will be saved. Set the name of our GCS below.

In [6]:
BUCKET_NAME = "gs://aliz_action_recognition_poc"  # @param {type:"string"}

**Only if the bucket doesn't already exist**: Run the following cell to create a new Cloud Storage bucket.

In [7]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://aliz_action_recognition_poc/...
ServiceException: 409 A Cloud Storage bucket named 'aliz_action_recognition_poc' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Validate access to the bucket by examining its contents:

In [8]:
! gsutil ls -al $BUCKET_NAME

 376743798  2022-03-31T17:13:44Z  gs://aliz_action_recognition_poc/Aspire_11_vs_AZ_Sky_11_Gold_2022-02-05.mp4#1648746824256424  metageneration=1
 367656178  2022-03-31T17:13:43Z  gs://aliz_action_recognition_poc/Aspire_15_vs_SG_Elite__2022-02-21.mp4#1648746823872950  metageneration=1
    101022  2022-04-21T14:59:17Z  gs://aliz_action_recognition_poc/Nagra - training.csv#1650553157400516  metageneration=1
 771812506  2022-03-22T07:48:19Z  gs://aliz_action_recognition_poc/RVA_16P_vs_OJVA.mp4#1647935299871660  metageneration=1
     34104  2022-03-23T18:21:05Z  gs://aliz_action_recognition_poc/sample.csv#1648059665898925  metageneration=1
       173  2022-03-31T18:52:01Z  gs://aliz_action_recognition_poc/test.json#1648752721716317  metageneration=1
       117  2022-03-31T18:47:27Z  gs://aliz_action_recognition_poc/test.jsonl#1648752447798314  metageneration=1
       168  2022-04-04T19:29:30Z  gs://aliz_action_recognition_poc/test_1.json#1649100570728275  metageneration=1
     77825  2022-0

### Vertex SDK Initialization

Initialize the Vertex SDK for Python for our project and corresponding bucket.

In [9]:
import google.cloud.aiplatform as aiplatform
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# Dataset Creation


## Labels 

Now set the variable `IMPORT_FILES` to the location of the CSV index files in Cloud Storage. 

In [10]:
IMPORT_FILES = [
    "gs://aliz_action_recognition_poc/training_expanded.csv",
]

Note: the csv file should have no header.

The columns should be as follows:
1. "test"/"train" 
2. URL of the video
3. Start time window for inspection
4. End time window for inspection
5. Name of action to be detected
6. Timestamp of the action

All timestamps should be specified as integers.

Let's take a quick pick at the import file specified above. 
- We count the number of examples by counting the number of rows in the CSV index file  (`wc -l`).
- We then peek at the first few rows.

In [11]:
FILE = IMPORT_FILES[0]

count = ! gsutil cat $FILE | wc -l
print("Number of Examples:", int(count[0]))

print("First 10 rows:")
! gsutil cat $FILE | head

Number of Examples: 893
First 10 rows:
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,365
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,380
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,401
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,427
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,459
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,497
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,535
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,568
train,gs://aliz_action_recognition_poc/trunc_video/Aspire_15_vs_SG_Elite__2022-02-21.mp4,0,1033,serve,604
train,g

## Creation of `Dataset` resource

Next, create the `Dataset` resource using the `create` method for the `VideoDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource. We'll use a combination of `TRAINING_TIMESTAMP` and `PREFIX`.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

This operation may take several minutes.

In [12]:
from datetime import datetime

TRAINING_TIMESTAMP = datetime.now().strftime("%Y/%m/%d_%H:%M:%S")

In [14]:
dataset = aiplatform.VideoDataset.create(
    display_name=PREFIX + TRAINING_TIMESTAMP,
    gcs_source=IMPORT_FILES,
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.action_recognition,
)

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating VideoDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create VideoDataset backing LRO: projects/276875326730/locations/us-central1/datasets/6750803482451640320/operations/120355360274907136
INFO:google.cloud.aiplatform.datasets.dataset:VideoDataset created. Resource name: projects/276875326730/locations/us-central1/datasets/6750803482451640320
INFO:google.cloud.aiplatform.datasets.dataset:To use this VideoDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.VideoDataset('projects/276875326730/locations/us-central1/datasets/6750803482451640320')
INFO:google.cloud.aiplatform.datasets.dataset:Importing VideoDataset data: projects/276875326730/locations/us-central1/datasets/6750803482451640320
INFO:google.cloud.aiplatform.datasets.dataset:Import VideoDataset data backing LRO: projects/276875326730/locations/us-central1/datasets/6750803482451640320/operations/809406103262593024
INFO:google.cloud

# Model training

To train an AutoML model, we will perform the following two steps:
1. create a training pipeline
2. run the pipeline.


## 1. Create training pipeline

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.

In [15]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name=PREFIX + TRAINING_TIMESTAMP,
    prediction_type="action_recognition",
)

print(job)

## 2. Run the training pipeline

Next, you run the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline for this project averages to 2 h 15 m.

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name=PREFIX + TRAINING_TIMESTAMP,
)

INFO:google.cloud.aiplatform.training_jobs:No dataset split provided. The service will use a default split.
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5080582097941299200?project=276875326730
INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/276875326730/locations/us-central1/trainingPipelines/5080582097941299200 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/276875326730/locations/us-central1/trainingPipelines/5080582097941299200 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/276875326730/locations/us-central1/trainingPipelines/5080582097941299200 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/276875326730/locations/us-central1/trainin

The evaluation of the trained model will be done in the next notebook.